In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6efbe5af92d01d26aaddfd8b7ad0355bbacb64b8c3d8f3558c79f5b11f7dd09b
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [1]:
pip install findspark 

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# değişkenleri uygun bir şekilde ayarlar

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark 
import pandas as pd 
import numpy as np
import seaborn as sns
from datetime import date, timedelta, datetime
import time
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark  
from pyspark.context import SparkContext
from pyspark.ml.feature import StringIndexer
from sklearn.metrics import f1_score, recall_score, precision_score
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, GBTClassifier, LinearSVC

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [ ]:
MAX_MEMORY = "10g"

spark = SparkSession \
    .builder \
    .appName("Churn") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [ ]:
df_csv=pd.read_csv("churn.csv")
df_csv

,Unnamed: 0,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,0,Cameron Williams,42.0,11066.80,0,7.22,8.0,1
1,1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,1
2,2,Eric Lozano,38.0,12884.75,0,6.67,12.0,1
3,3,Phillip White,42.0,8010.76,0,6.71,10.0,1
4,4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,1
...,...,...,...,...,...,...,...,...
895,895,Paul Miller,42.0,12800.82,1,3.62,8.0,0
896,896,Natalie Hodges,52.0,9893.92,0,6.91,7.0,0
897,897,Ana Smith,45.0,12056.18,0,5.46,4.0,0
898,898,Justin Leonard,51.0,6517.93,1,5.47,10.0,0


In [ ]:
df=spark.read.csv("churn.csv",header=True,sep=",",inferSchema=True)

In [ ]:
df.show()

+---+-------------------+----+--------------+---------------+-----+---------+-----+
|_c0|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+-------------------+----+--------------+---------------+-----+---------+-----+
|  0|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|     Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
|  5|   Jessica Williams|48.0|      10356.02|              0| 5.12|      8.0|    1|
|  6|        Eric Butler|44.0|      11331.58|              1| 5.23|     11.0|    1|
|  7|      Zachary Walsh|32.0|       9885.12|              1| 6.92|      9.0|    1|
|  8|        Ashlee Carr|43.0|       14062.6|              1| 5.46|     11.0

In [ ]:
df.select("Names").dropDuplicates().count()

899

In [ ]:
df.select('Age').distinct().collect()

[Row(Age=49.0),
 Row(Age=29.0),
 Row(Age=47.0),
 Row(Age=42.0),
 Row(Age=44.0),
 Row(Age=35.0),
 Row(Age=39.0),
 Row(Age=37.0),
 Row(Age=34.0),
 Row(Age=25.0),
 Row(Age=36.0),
 Row(Age=41.0),
 Row(Age=56.0),
 Row(Age=50.0),
 Row(Age=45.0),
 Row(Age=31.0),
 Row(Age=58.0),
 Row(Age=51.0),
 Row(Age=48.0),
 Row(Age=22.0),
 Row(Age=60.0),
 Row(Age=53.0),
 Row(Age=46.0),
 Row(Age=28.0),
 Row(Age=38.0),
 Row(Age=27.0),
 Row(Age=40.0),
 Row(Age=30.0),
 Row(Age=33.0),
 Row(Age=52.0),
 Row(Age=32.0),
 Row(Age=55.0),
 Row(Age=26.0),
 Row(Age=65.0),
 Row(Age=54.0),
 Row(Age=43.0)]

In [ ]:
df.groupBy("names").count().sort("count",ascending=False).show(10)

+-----------------+-----+
|            names|count|
+-----------------+-----+
|    Jennifer Wood|    2|
|     Patrick Bell|    1|
|    Chelsea Marsh|    1|
| Patrick Robinson|    1|
|Mr. Jerome Dawson|    1|
|      John Barber|    1|
|      Amber Evans|    1|
|    David Compton|    1|
|       Lisa Davis|    1|
|    Maria Stanley|    1|
+-----------------+-----+
only showing top 10 rows



In [ ]:
df.filter(df["names"]=="Jennifer Wood").show() 
#JENNİFER WOOD isimli kişiler aynı mı yoksa farklı mı ona bakıldı ve farklı kişiler olduğunu gördük

+---+-------------+----+--------------+---------------+-----+---------+-----+
|_c0|        Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+-------------+----+--------------+---------------+-----+---------+-----+
| 22|Jennifer Wood|35.0|       9381.12|              1| 6.78|     11.0|    1|
|439|Jennifer Wood|48.0|      11585.16|              0| 4.61|      9.0|    0|
+---+-------------+----+--------------+---------------+-----+---------+-----+



In [ ]:
df.select('age','total_purchase','account_manager','years','num_sites','churn').describe().show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|              age|   total_purchase|   account_manager|            years|         num_sites|              churn|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|              900|              900|               900|              900|               900|                900|
|   mean|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|             22.0|            100.0|                 0|              1.0|               3.0|                  0|
|    max|             65.0|         18026.01|                 1|             9.15|              14.0|                  1|
+-------+---------------

In [ ]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
_c0,900,449.5,259.95191863111916,0,899
Names,900,None,None,Aaron King,Zachary Walsh
Age,900,41.81666666666667,6.127560416916251,22.0,65.0
Total_Purchase,900,10062.82403333334,2408.644531858096,100.0,18026.01
Account_Manager,900,0.4811111111111111,0.4999208935073339,0,1
Years,900,5.27315555555555,1.274449013194616,1.0,9.15
Num_Sites,900,8.587777777777777,1.7648355920350969,3.0,14.0
Churn,900,0.16666666666666666,0.3728852122772358,0,1


In [ ]:
graphic_df=df.drop("index","names").toPandas()

In [ ]:
df=df.dropna()

In [ ]:
df=df.withColumn("age_sqrt",df.Age**2)

In [ ]:
stringIndexer=StringIndexer(inputCol="Churn",outputCol="label")

In [ ]:
indexed=stringIndexer.fit(df).transform(df)

In [ ]:
indexed.dtypes

[('_c0', 'int'),
 ('Names', 'string'),
 ('Age', 'double'),
 ('Total_Purchase', 'double'),
 ('Account_Manager', 'int'),
 ('Years', 'double'),
 ('Num_Sites', 'double'),
 ('Churn', 'int'),
 ('age_sqrt', 'double'),
 ('label', 'double')]

In [ ]:
df=indexed.withColumn("label",indexed.label.cast("integer"))

In [ ]:
free_val=['Age','Total_Purchase','Account_Manager','Years','Num_Sites']

In [ ]:
vectorAssembler=VectorAssembler(inputCols=free_val,outputCol="features")

In [ ]:
features_val=vectorAssembler.transform(df)

In [ ]:
res_df=features_val.select("features","label")
res_df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
res_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)



In [ ]:
splits=res_df.randomSplit([0.7,0.3])

In [ ]:
train_df=splits[0]
test_df=splits[1]

In [ ]:
gbtc=GBTClassifier(maxIter=10,featuresCol="features",labelCol="label")

In [ ]:
gbtc_model=gbtc.fit(train_df)

In [ ]:
y_pred=gbtc_model.transform(test_df)

In [ ]:
y_pred.show(10)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[1.33410049929913...|[0.93512397654654...|       0.0|
|[27.0,8628.8,1.0,...|    0|[1.24164894235166...|[0.92296261345853...|       0.0|
|[28.0,11204.23,0....|    0|[0.93855947130123...|[0.86727985164421...|       0.0|
|[29.0,12711.15,0....|    0|[1.26401863859851...|[0.92608411275730...|       0.0|
|[29.0,13255.05,1....|    0|[1.33410049929913...|[0.93512397654654...|       0.0|
|[30.0,6744.87,0.0...|    0|[1.08004585110755...|[0.89660804983470...|       0.0|
|[30.0,7960.64,1.0...|    1|[1.01399648049924...|[0.88370497233427...|       0.0|
|[30.0,8403.78,1.0...|    0|[1.32641084435195...|[0.93418469093225...|       0.0|
|[30.0,8677.28,1.0...|    0|[1.30640209114811...|[0.93168110268566...|       0.0|
|[30.0,10183.98,

In [ ]:
ac=y_pred.select("label","prediction")
ac.show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



In [ ]:
ac.filter(ac.label==ac.prediction).count()/ac.count()

0.8528301886792453

In [ ]:
evaluator=BinaryClassificationEvaluator()
pramGrid=(ParamGridBuilder()
         .addGrid(gbtc.maxDepth,[2,4,6])
         .addGrid(gbtc.maxBins,[20,30])
         .addGrid(gbtc.maxIter,[10,20])
         .build())

cv=CrossValidator(estimator=gbtc,estimatorParamMaps=pramGrid,evaluator=evaluator,numFolds=10)

In [ ]:
cv_model=cv.fit(train_df)
y_pred=cv_model.transform(test_df)

In [ ]:
y_pred.show(10)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[1.43308494901862...|[0.94614853173913...|       0.0|
|[27.0,8628.8,1.0,...|    0|[1.37216158003068...|[0.93959194142909...|       0.0|
|[28.0,11204.23,0....|    0|[0.78052778673763...|[0.82650476872659...|       0.0|
|[29.0,12711.15,0....|    0|[1.35149309279572...|[0.93720262213398...|       0.0|
|[29.0,13255.05,1....|    0|[1.43308494901862...|[0.94614853173913...|       0.0|
|[30.0,6744.87,0.0...|    0|[1.05083204306039...|[0.89106481398323...|       0.0|
|[30.0,7960.64,1.0...|    1|[0.88810936088568...|[0.85522932638015...|       0.0|
|[30.0,8403.78,1.0...|    0|[1.45375343625358...|[0.94821628328817...|       0.0|
|[30.0,8677.28,1.0...|    0|[1.18577731017089...|[0.91463232364787...|       0.0|
|[30.0,10183.98,

In [ ]:
ac=y_pred.select("label","prediction")

In [ ]:
ac.filter(ac.label==ac.prediction).count()/ac.count()

0.8830188679245283

In [ ]:
evaluator.evaluate(y_pred)

0.8720371556694425